In [1]:
from __future__ import absolute_import, print_function
import time
import numpy as np
import pyopencl as cl
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from scipy import signal
from matplotlib import rcParams
from scipy import io
import os

import locale
os.environ["PYTHONIOENCODING"] = "utf-8"
# scriptLocale=locale.setlocale(category=locale.LC_ALL, locale="en_GB.UTF-8")

In [2]:
# Widening the screen
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

rcParams['figure.dpi'] = 600
rcParams['font.size'] = 16
rcParams['font.family'] = 'StixGeneral'
plt.rc('font', size=16)          # controls default text sizes
plt.rc('axes', titlesize=16)     # fontsize of the axes title
plt.rc('axes', labelsize=16)     # fontsize of the x and y labels
plt.rc('xtick', labelsize=16)    # fontsize of the tick labels
plt.rc('ytick', labelsize=16)    # fontsize of the tick labels
plt.rc('legend', fontsize=16)    # legend fontsize
plt.rc('figure', titlesize=16)   # fontsize of the figure title

### Simulation settings

In [3]:
length    = 1024.0       # 2.5 Length of the physical landscape
n         = 1024         # Size of the 2D grid
endtime   = 2000.0       # end time
dT        = 0.02         # 0.02     #calculate time step
nplot     = 100
nsteps    = np.ceil(endtime/nplot/dT).astype(int)    #number of time steps

### Precalculations

In [4]:
dX = 1.0 #length/n      # Spatial step size
dY = 1.0 #length/n      # Spatial step size
Grid_Width  = n
Grid_Height = n
# %% Reporting in the simulation on the console
print(" Current grid dimensions: %d x %d cells\n" % (Grid_Width, Grid_Height));

 Current grid dimensions: 1024 x 1024 cells



### Defining the device that is used

In [5]:
# Setting up the OpenCL context
DeviceNr = 0   # 0 = GTX 960M
platform = cl.get_platforms()
Devices  = platform[0].get_devices()  # 0 = GPU
context  = cl.Context([Devices[DeviceNr]])
queue    = cl.CommandQueue(context)
mf       = cl.mem_flags # Memory flags are set
print(" Compute Device: %s\n" % Devices[DeviceNr].name)

 Compute Device: Intel(R) Core(TM) i7-10700K CPU @ 3.80GHz



### Parameter definition 

In [6]:
#parameter
a=4.0;    
b=1.0;
T=7.0;
Da=1/16;
Db=1.0;
DeltaMin=1.0/50.0
DeltaMax=1./2.0
BioMax=20.0
BioMin=1.0

In [7]:
np.mean(np.mean(np.random.rand(n*n)))

0.499637635927418

### Initial values

In [8]:
# The array is allocated on the GPU and the initial values are copied onto it
T        = np.tile(np.logspace(np.log10(BioMin),np.log10(BioMax),n),n) #np.tile(np.linspace(BioMin,BioMax,n),n) # total biomass X-axis
A        = 0.4*T + (np.random.rand(n*n)-0.5)*0.1
#(np.sqrt(a)-b + np.sqrt(a/c)-b)/2 + (np.random.rand(n*n)-0.5)
A_host   = A.astype(np.float32)
A_g      = cl.Buffer(context, mf.READ_WRITE | mf.COPY_HOST_PTR, hostbuf=A_host)

B        = 0.6*T + (np.random.rand(n*n)-0.5)*0.0
B_host   = B.astype(np.float32)
B_g      = cl.Buffer(context, mf.READ_WRITE | mf.COPY_HOST_PTR, hostbuf=B_host)

#Da array -> GPU diffusion Da/Db=Delta
# np.logspace(np.log10(Par_max),np.log10(Par_min),n).repeat(n)
Da_arr        =  np.logspace(np.log10(DeltaMin),np.log10(DeltaMax),n).repeat(n) #np.linspace(DeltaMax,DeltaMin,n).repeat(n) # y-axis
Da_arr_host   = Da_arr.astype(np.float32)
Da_arr_g      = cl.Buffer(context, mf.READ_WRITE | mf.COPY_HOST_PTR, hostbuf=Da_arr_host)


In [9]:
# plt.imshow(Da_arr_host.reshape(Grid_Width, Grid_Height),cmap='YlGn',extent=[0,length,0,length])
# Xlabels = np.linspace(0,length,4).round(1)
# Ylabels = np.linspace(0,length,4).round(1)
# plt.yticks(Ylabels,color='black',rotation=90) 
# plt.xticks(Xlabels,color='black',rotation=0) 
# cbar=plt.colorbar(fraction=0.046,pad=0.04);


In [10]:
T_host   = T.astype(np.float32)
T_g      = cl.Buffer(context, mf.READ_WRITE | mf.COPY_HOST_PTR, hostbuf=T_host)

cl.enqueue_copy(queue, T_host, T_g)
# plt.imshow(A_host.reshape(Grid_Width, Grid_Height),cmap='YlGn',extent=[0,length,0,length])
# cbar=plt.colorbar(fraction=0.046,pad=0.04);

In [11]:
#Da is X or Y ?
cl.enqueue_copy(queue, Da_arr_host, Da_arr_g)
# plt.imshow(Da_arr_host.reshape(Grid_Width, Grid_Height),cmap='YlGn',extent=[0,length,0,length])
# cbar=plt.colorbar(fraction=0.046,pad=0.04);

In [12]:
with open('SpatialFunctions_iPy.cl', 'r',encoding='utf-8') as myfile:
   SpatialFunctions = myfile.read()

### Defining the OpenCL simulation kernel

In [13]:
# List of parameters for the OpenCL kernel. Seperate with comma without spaces
# Setting up the parameters for the Kernel

PassVars="Da,a,b,Db,dX,dY,dT,Grid_Width,Grid_Height"

PassVals=eval(PassVars)
PassVars=PassVars.split(',')
Params=""

for ii in range(len(PassVals)):
    Params = Params+"#define " + PassVars[ii] + " " + str(PassVals[ii]) + " \n"

In [14]:
ComputeCode = """

///////////////////////////////////////////////////////////////////////////////
// Simulation kernel
///////////////////////////////////////////////////////////////////////////////   

__kernel void SimulationKernel (__global float* A, __global float* B, __global float* Da_arr)
{
    size_t current  = get_global_id(0);
    size_t row      = floor((float)current/(float)Grid_Width);
    size_t column   = current%Grid_Width;

        if (row > 0 && row < Grid_Width-1 && column > 0 && column < Grid_Height-1)
            {
             float dAdt =   A[current]*B[current] - a*A[current]/(A[current]+b) + Da_arr[current]*d2_dxy2(A);
             float dBdt =   a*A[current]/(A[current]+b) - A[current]*B[current] + Db*d2_dxy2(B);
             
             A[current] = A[current] + dAdt*dT;
             B[current] = B[current] + dBdt*dT;
            }
            
        // HANDLE Boundaries
        else 
            {
             NeumannBoundaries(A);
             NeumannBoundaries(B);
            }

} // End SimulationKernel
"""

### Here the kernel is compiled

In [15]:
program = cl.Program(context, Params + SpatialFunctions + ComputeCode).build()

In [ ]:
from ipywidgets import FloatProgress
from IPython.display import display

# Setting up a progress bar for the simulation
print("Progress :");
PB = FloatProgress(min=0, max=nplot); display(PB) 

As=np.zeros((Grid_Width, Grid_Height, nplot))
Bs=np.zeros((Grid_Width, Grid_Height, nplot))

# Set up simulation parameters
global_size = A_host.shape

# Start the timer:
start_time = time.time()

# Starting the loop
for ii in range(1,nplot):
    # The simulation
    for jj in range(1,nsteps):      
        program.SimulationKernel(queue, global_size, None, A_g, B_g, Da_arr_g)

    # Get the data from the GPU
    cl.enqueue_copy(queue, A_host, A_g)
    cl.enqueue_copy(queue, B_host, B_g)

    # We store the state of the system for <NumPlot> different times.
    As[:,:,ii] = A_host.reshape(Grid_Width, Grid_Height)
    Bs[:,:,ii] = B_host.reshape(Grid_Width, Grid_Height)
    print(ii,end=" ")
#     PB.value += 1 # signal to increment the progress bar

# Determining the time that we used for the simulation
elapsed_time = time.time() - start_time    
print(" Simulation took      : %1.1f (s)" % (elapsed_time))


Progress :


FloatProgress(value=0.0)

/opt/anaconda3/lib/python3.8/site-packages/pytools/persistent_dict.py:484: UserWarning: could not obtain lock--delete '/Users/qxliu/Library/Caches/pytools/pdict-v4-pyopencl-invoker-cache-v7-py3.8.5.final.0/72a18d6fef0605601ff4ca0da8675e46dcfc02fab64df8ccc8361db26cfd1bbe.lock' if necessary
  self.store(key, value, _skip_if_present=True, _stacklevel=1 + _stacklevel)


In [ ]:
from scipy import io
import os
if not os.path.exists("data"):
    os.makedirs("data")
io.savemat('data/UV_data.mat', {'A': As, 'B': Bs, 'Detlta': Da_arr, 'Tau': T})

# load data: mathfn.mat from Matlab
# data = io.loadmat(matfn) 

# Plotting the results

In [ ]:
### The Simulation loop
nbin=4
# viridis, Reds, Greens, YlGn
from matplotlib import ticker
fig, ax = plt.subplots(1, 2, figsize=(10, 5))
im0 = ax[0].imshow(A_host.reshape(n,n),interpolation='nearest', cmap='YlGn',
                   extent=[0,length,0,length],clim=(0,BioMax));#, clim=(0,0.1));
ax[0].set_title('U density (g/m$^2$)');
cbar=plt.colorbar(im0, ax=ax[0],fraction=0.046,pad=0.04);
tick_locator = ticker.MaxNLocator(nbins=5)
cbar.locator = tick_locator
cbar.update_ticks()
# Xlabels = np.linspace(BioMin,BioMax,nbin).round(1)
Xlabels = np.logspace(np.log10(BioMin),np.log10(BioMax),nbin).round(0)
Ylabels = np.linspace(1.0/DeltaMax,1.0/DeltaMin,nbin).round(0)
ax[0].set_xticks(np.linspace(0,length,nbin))
ax[0].set_xticklabels(Xlabels)
ax[0].set_yticks(np.linspace(0,length,nbin))
ax[0].set_yticklabels(Ylabels)
ax[0].set_xlabel('global density, $\\tau$');
ax[0].set_ylabel('diffusion ratio, $\delta$');

im1 = ax[1].imshow(B_host.reshape(n,n),interpolation='nearest',cmap='YlGn',
                   extent=[0,length,0,length],clim=(0,2));#, clim=(0,0.5));
ax[1].set_title('V density (g/m$^2$)');
cbar=plt.colorbar(im1, ax=ax[1],fraction=0.046,pad=0.04);
tick_locator = ticker.MaxNLocator(nbins=5)
cbar.locator = tick_locator
cbar.update_ticks()
# Xlabels = np.linspace(BioMin,BioMax,nbin).round(0)
Xlabels = np.logspace(np.log10(BioMin),np.log10(BioMax),nbin).round(0)
Ylabels = np.linspace(1.0/DeltaMax,1.0/DeltaMin,nbin).round(0)
ax[1].set_xticks(np.linspace(0,length,nbin))
ax[1].set_xticklabels(Xlabels)
ax[1].set_yticks(np.linspace(0,length,nbin))
ax[1].set_yticklabels(Ylabels)
ax[1].set_xlabel('global density, $\\tau$');
# ax[1].set_ylabel('diffusion ratio, $\delta$');


fig.tight_layout()

text=fig.suptitle("Time: %1.0f of %1.0f" % (endtime, endtime), x=0.5, y=0.00, fontsize=16);

# Making a movie

In [ ]:
from matplotlib import animation, rc
if not os.path.exists("Images"):
    os.makedirs("Images")
    
def updatefig(i): # To update the image at each iteration
    global As, Bs
    im0.set_array(As[:,:,i])
    im1.set_array(Bs[:,:,i])
    text.set_text("Time: %1.0f of %1.0f" % ((i+1)/nplot*endtime, endtime));
    fig.savefig('Images/Images %03g'% i+'.pdf',bbox_inches='tight',dpi=600)
    return im0,im1,

ani = animation.FuncAnimation(fig, updatefig, interval=100, frames = nplot, repeat=False, blit=True)
print("Save figure finshes!")

In [ ]:
# print('time%03g'% 5+"test")

In [ ]:
from IPython.display import HTML
HTML(ani.to_html5_video()) 
# write to an mp4 movie
Writer = animation.writers['ffmpeg']
writer = Writer(fps=5, bitrate=1800)
ani.save('PSModel_scaled_T_Da.mp4', writer=writer)
print("Save movies finshes!")

## The End, Kang. 2020.0709 ECNU